In [1]:
from pprint import pprint
import base64
import json
import mimetypes
import os
import requests
import sys
import pandas as pd
from openai import OpenAI

# Prepare images to call gpt api

In [2]:
master_df = pd.read_csv("../../master_list.csv")
print(master_df.shape)
master_df.head()

(4454, 12)


,filepath,xmin,ymin,xmax,ymax,label,ProductType,Weight,HalalStatus,HealthStatus,new_camera,tag
0,IMG_20230428_123528_jpg.rf.5687b7b914f6d9aa98c...,151,42,497,591,Sugar_400-499g_NonHalal_NonHealthy,Sugar,400-499g,NonHalal,NonHealthy,0,NaN
1,IMG_20230428_123522_jpg.rf.204ff37f497f2dce442...,88,81,442,567,Sugar_400-499g_NonHalal_NonHealthy,Sugar,400-499g,NonHalal,NonHealthy,0,NaN
2,IMG_20230428_123708_jpg.rf.141ecd0cefaea75c0b7...,35,34,492,622,Sugar_400-499g_NonHalal_NonHealthy,Sugar,400-499g,NonHalal,NonHealthy,0,NaN
3,IMG_20230428_123521_jpg.rf.1069b402272252862ec...,99,122,428,587,Sugar_400-499g_NonHalal_NonHealthy,Sugar,400-499g,NonHalal,NonHealthy,0,NaN
4,IMG_20230428_123659_jpg.rf.5e1b6c4caabe48cf360...,103,17,474,592,Sugar_400-499g_NonHalal_NonHealthy,Sugar,400-499g,NonHalal,NonHealthy,0,NaN


In [3]:
prd_group_df = pd.read_csv("../product_group.csv")
prd_group_df['group'] = prd_group_df['group'].astype('Int64')
prd_group_df = prd_group_df.loc[prd_group_df['tag'] != "test",]
prd_group_df.head()

,filepath,label,ProductType,Weight,HalalStatus,HealthStatus,new_camera,tag,group
0,IMG_20230428_123528_jpg.rf.5687b7b914f6d9aa98c...,Sugar_400-499g_NonHalal_NonHealthy,Sugar,400-499g,NonHalal,NonHealthy,0,NaN,1
1,IMG_20230428_123522_jpg.rf.204ff37f497f2dce442...,Sugar_400-499g_NonHalal_NonHealthy,Sugar,400-499g,NonHalal,NonHealthy,0,NaN,1
2,IMG_20230428_123708_jpg.rf.141ecd0cefaea75c0b7...,Sugar_400-499g_NonHalal_NonHealthy,Sugar,400-499g,NonHalal,NonHealthy,0,NaN,2
3,IMG_20230428_123521_jpg.rf.1069b402272252862ec...,Sugar_400-499g_NonHalal_NonHealthy,Sugar,400-499g,NonHalal,NonHealthy,0,NaN,1
4,IMG_20230428_123659_jpg.rf.5e1b6c4caabe48cf360...,Sugar_400-499g_NonHalal_NonHealthy,Sugar,400-499g,NonHalal,NonHealthy,0,NaN,2


In [4]:
total_df = pd.merge(master_df, prd_group_df[['filepath','group']], on='filepath', how='left')
total_df = total_df.dropna(subset=['group'])
total_df = total_df.reset_index(drop=True)
print(total_df.shape)
total_df.head()

(2947, 13)


,filepath,xmin,ymin,xmax,ymax,label,ProductType,Weight,HalalStatus,HealthStatus,new_camera,tag,group
0,IMG_20230428_123528_jpg.rf.5687b7b914f6d9aa98c...,151,42,497,591,Sugar_400-499g_NonHalal_NonHealthy,Sugar,400-499g,NonHalal,NonHealthy,0,NaN,1
1,IMG_20230428_123522_jpg.rf.204ff37f497f2dce442...,88,81,442,567,Sugar_400-499g_NonHalal_NonHealthy,Sugar,400-499g,NonHalal,NonHealthy,0,NaN,1
2,IMG_20230428_123708_jpg.rf.141ecd0cefaea75c0b7...,35,34,492,622,Sugar_400-499g_NonHalal_NonHealthy,Sugar,400-499g,NonHalal,NonHealthy,0,NaN,2
3,IMG_20230428_123521_jpg.rf.1069b402272252862ec...,99,122,428,587,Sugar_400-499g_NonHalal_NonHealthy,Sugar,400-499g,NonHalal,NonHealthy,0,NaN,1
4,IMG_20230428_123659_jpg.rf.5e1b6c4caabe48cf360...,103,17,474,592,Sugar_400-499g_NonHalal_NonHealthy,Sugar,400-499g,NonHalal,NonHealthy,0,NaN,2


In [5]:
result_df = total_df.groupby(['label', 'group'])['filepath'].first().reset_index()
print(result_df.shape)
result_df.head()

(351, 3)


,label,group,filepath
0,AdultMilk_1-99g_Halal_NonHealthy,1,2023_10_25_11_23_10_398347.jpg
1,AdultMilk_1-99g_Halal_NonHealthy,2,2023_10_25_11_29_12_163138.jpg
2,AdultMilk_1-99g_Halal_NonHealthy,3,2023_10_25_11_30_13_941552.jpg
3,AdultMilk_1000-1999g_Halal_NonHealthy,1,2023_10_25_11_18_17_414290.jpg
4,AdultMilk_1000-1999g_Halal_NonHealthy,2,2023_10_25_11_24_30_751596.jpg


In [6]:
total_df['ProductType'].unique()

array(['Sugar', 'OtherNoodles', 'Oil', 'BiscuitsCrackersCookies', 'Nuts',
       'HoneyOtherSpreads', 'BeehoonVermicelli', 'MiloPowder',
       'RolledOatsInstantOatmeal', 'OtherSauceDressing',
       'SweetsChocolatesOthers', 'BreakfastCereals', 'Coffee',
       'InstantNoodles', 'CerealBeveragePowder', 'OtherBakingNeeds',
       'BabyMilkPowder', 'MaternalMilkPowder', 'TeaPowderLeaves', 'Pasta',
       'FlavoredMilk', 'Sardines', 'Flour', 'Babyfood', 'Potatochips',
       'Salt', 'Peanutbutter', 'ChilliSauce', 'NutellaChocolate',
       'InstantMeals', 'Kaya', 'AdultMilk', 'CannedPacketCreamersSweet',
       'CornChip', 'SoftDrinks', 'OtherDriedFood', 'FruitJuice',
       'CannedBakedBeans', 'CannedBeefOtherMeats', 'CannedBraisedPeanuts',
       'CannedChicken', 'CannedFruitsCocktail', 'CannedMashrooms',
       'CannedPickles', 'CannedPorkLunchronMeat', 'CannedSardingMackerel',
       'CannedSoup', 'CannedTunaDace', 'CannedVegetarianFood',
       'ChocolateMaltPowder', 'CoffeePowder'

In [7]:
total_df['Weight'].unique()

array(['400-499g', '700-799g', '500-599g', '200-299g', '100-199g',
       '1-99g', '300-399g', '600-699g', '800-899g', '1000-1999g',
       '900-999g', '3000-3999g'], dtype=object)

In [8]:
total_df['HalalStatus'].unique()

array(['NonHalal', 'Halal'], dtype=object)

In [9]:
total_df['HealthStatus'].unique()

array(['NonHealthy', 'Healthy'], dtype=object)

# Call gpt to recognize product, weight, halal and health

In [15]:
import base64

def encode_image(image_path):
    """Encodes an image to Base64."""
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')


In [16]:
with open('/Users/liupeng/Desktop/Research/api.txt', 'r') as file:
    api_key = file.read()

In [17]:
exisitng_paths = []

In [18]:
image_paths = total_df['filepath']
new_paths = [i for i in image_paths if i not in exisitng_paths]

In [ ]:
# Placeholder for the parsed data
parsed_data = []

# Iterate over each image path
for tmp_img_path in new_paths:
    # if tmp_img_path != new_paths[0]:
    #     break
    image_path = os.path.join("../../../rshiny/www/all_images/", tmp_img_path)
    base64_image = encode_image(image_path)
    
    headers = {
          "Content-Type": "application/json",
          "Authorization": f"Bearer {api_key}"
        }

    payload = {
              "model": "gpt-4-vision-preview",
              "messages": [
                {
                  "role": "user",
                  "content": [
                    {
                      "type": "text",
                      "text": """ 
                      For this image, can you a prediction for the following four labels? 
                      The product type is based on the appearance of the product, 
                      the weight is to recognize how heavy the product is by identify the weight information on the appearance,
                      the halal status is to recognize if the product is halal food or not, 
                      and the healthy status is to recognize if the product is healthy if it contains a read triangle shape based on Singapore standard.
                      Note that you can only choose one from the given options in the bracket for each label.
                        product type (
                            'Sugar', 'OtherNoodles', 'Oil', 'BiscuitsCrackersCookies', 'Nuts',
                           'HoneyOtherSpreads', 'BeehoonVermicelli', 'MiloPowder',
                           'RolledOatsInstantOatmeal', 'OtherSauceDressing',
                           'SweetsChocolatesOthers', 'BreakfastCereals', 'Coffee',
                           'InstantNoodles', 'CerealBeveragePowder', 'OtherBakingNeeds',
                           'BabyMilkPowder', 'MaternalMilkPowder', 'TeaPowderLeaves', 'Pasta',
                           'FlavoredMilk', 'Sardines', 'Flour', 'Babyfood', 'Potatochips',
                           'Salt', 'Peanutbutter', 'ChilliSauce', 'NutellaChocolate',
                           'InstantMeals', 'Kaya', 'AdultMilk', 'CannedPacketCreamersSweet',
                           'CornChip', 'SoftDrinks', 'OtherDriedFood', 'FruitJuice',
                           'CannedBakedBeans', 'CannedBeefOtherMeats', 'CannedBraisedPeanuts',
                           'CannedChicken', 'CannedFruitsCocktail', 'CannedMashrooms',
                           'CannedPickles', 'CannedPorkLunchronMeat', 'CannedSardingMackerel',
                           'CannedSoup', 'CannedTunaDace', 'CannedVegetarianFood',
                           'ChocolateMaltPowder', 'CoffeePowder', 'CookingCreamMilk',
                           'CurryPastePowderSauce', 'DarkSoySauce', 'DriedBeans', 'DriedFood',
                           'DriedFruits', 'DriedMeatSeafood', 'DriedVegetables',
                           'HeabsSpicesPepper', 'Jam', 'KetchupChillSauce', 'LightSoySauce',
                           'MilkDrink', 'MilkPowder', 'OtherCannedBeansPeasPeanuts',
                           'OtherCannedSeafood', 'OtherCannedVegetables', 'OtherHotBeverages',
                           'OtherSpreads', 'PastaSauce', 'SoftDrinksOtherReadyToDrink',
                           'SoupStock', 'TeaDrink'
                        ),
                        weight ('400-499g', '700-799g', '500-599g', '200-299g', '100-199g',
                           '1-99g', '300-399g', '600-699g', '800-899g', '1000-1999g',
                           '900-999g', '3000-3999g'
                        ),
                        halal status ('NonHalal', 'Halal'),
                        healthy status ('NonHealthy', 'Healthy'),
                        
                        Also, provide the following assessment
                        image reflection (High, Medium, Low)
                        image clarity (High, Medium, Low)
                        prediction confidence for the product type (High, Medium, Low)
                        prediction confidence for the weight (High, Medium, Low)
                        prediction confidence for the halal status (High, Medium, Low)
                        prediction confidence for the healthy status (High, Medium, Low)

                        format your answer in json format so that it could be easily converted to dataframe
                        """
                    },
                    {
                      "type": "image_url",
                      "image_url": {
                        "url": f"data:image/jpeg;base64,{base64_image}"
                      }
                    }
                  ]
                }
              ],
              "max_tokens": 300
            }


    response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, json=payload)
    content = response.json()['choices'][0]['message']['content']

    response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, json=payload)

    # Extracting the message content (the table with predictions)
    content = response.json()['choices'][0]['message']['content']

    # Parsing the content to extract prediction values
    # Splitting the content string into lines and then parsing each line
    lines = content.strip().split('\n')[2:]  # Skipping the header

    # Parsing the strings to extract the relevant information
    data_parsed = [line.replace('"', '').strip() for line in lines if ':' in line]
    data_dict = {item.split(":")[0].strip(): item.split(":")[1].strip().strip(',') for item in data_parsed}

    # Converting the dictionary into a DataFrame
    df_from_strings = pd.DataFrame([data_dict])

    parsed_data.append(df_from_strings)


In [41]:
parsed_data

[  product type    weight halal status healthy status image reflection  \
 0        Sugar  300-399g        Halal     NonHealthy           Medium   
 
   image clarity prediction confidence for the product type  \
 0          High                                       High   
 
   prediction confidence for the weight  \
 0                                 High   
 
   prediction confidence for the halal status  \
 0                                     Medium   
 
   prediction confidence for the healthy status  
 0                                         High  ]

In [54]:
col_names = ['product_type', 'weight', 'halal', 'health', 'image_reflection', 
                  'image_clarity', 'product_type_confidence', 'weight_confidence',
                  'halal_confidence', 'health_confidence']

# Normalizing column names for consistency across all DataFrames
normalized_dfs = []
# for df in parsed_data:
for i in range(len(parsed_data)):
    df = parsed_data[i]
    if df.shape[1] == 11:
        df = df.iloc[:,1:]
    # Renaming columns to have consistent names across all DataFrames
    df.columns = col_names
    normalized_dfs.append(df)

# Combining all DataFrames into a single DataFrame
combined_df = pd.concat(normalized_dfs, ignore_index=True)

# Add img filename
combined_df['img_filename'] = image_paths.tolist()[:len(combined_df)]
cols = combined_df.columns.tolist()
cols = cols[-1:] + cols[:-1]  # Move the last column to the first position
combined_df = combined_df[cols] 

combined_df

,img_filename,product_type,weight,halal,health,image_reflection,image_clarity,product_type_confidence,weight_confidence,halal_confidence,health_confidence
0,IMG_20230428_123528_jpg.rf.5687b7b914f6d9aa98c...,Sugar,300-399g,Halal,NonHealthy,Medium,High,High,High,Medium,High


In [55]:
combined_df['img_filename'][0]

'IMG_20230428_123528_jpg.rf.5687b7b914f6d9aa98cadf060d1e3b00.jpg'

In [13]:
combined_df.to_csv("chatgpt/gpt_rst_286-313.csv")

# Add imgs where big model predicts wrongly

In [27]:
combined_df.to_csv("chatgpt/gptpred_bigmodelwrong.csv")

# Combine all gpt predictions

In [28]:
import pandas as pd
import glob

In [29]:
path = 'chatgpt' # Update this to your folder's path
all_files = glob.glob(path + "/*.csv")
all_files

['chatgpt/gpt_rst_116-160.csv',
 'chatgpt/gpt_rst_286-313.csv',
 'chatgpt/gpt_rst_161-197.csv',
 'chatgpt/gpt_rst_231-246.csv',
 'chatgpt/gpt_rst_246-272.csv',
 'chatgpt/gptpred_bigmodelwrong.csv',
 'chatgpt/gpt_rst_0-115.csv',
 'chatgpt/gpt_rst_272-286.csv',
 'chatgpt/gpt_rst_198-230.csv']

In [30]:
all_dataframes = [] # A list to store each dataframe

for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=0)
    all_dataframes.append(df)

combined_df = pd.concat(all_dataframes, axis=0, ignore_index=True)


In [31]:
final_df = combined_df.drop_duplicates()
final_df = final_df.iloc[:,1:]
final_df.head()

,img_filename,product_type,image_reflection,image_clarity,prediction_confidence
0,IMG_0184_jpeg.rf.446d62343e17b8ef909b6f0ff4530...,OtherNoodles,Medium,High,Low
1,IMG_0118_jpeg.rf.305bb0eadb135b869c11332c7173a...,OtherSauceDressingSoupbasePaste,Low,High,High
2,IMG_0096_jpeg.rf.4b1ab7e2cd4c285bd9e74d1ec1ab0...,Book,Low,Medium,Medium
3,IMG_0105_jpeg.rf.62a7a2c99f2d635525abcca89cb9f...,OtherSauceDressingSoupbasePaste,Low,Medium,Low
4,IMG_0124_jpeg.rf.3d66e8ddcd7d9d088875a3e2cf365...,OtherDriedFood,Low,High,High


In [32]:
final_df.shape

(350, 5)

In [33]:
final_df.to_csv("chatgpt_prediction.csv")